In [2]:
import requests
import sys
import time
import json
from datetime import datetime, timedelta
from typing import Dict, List, Tuple
import pandas as pd
import os, re
from glob import glob

In [ ]:
url = "https://giris.epias.com.tr/cas/v1/tickets"
headers = {"Accept": "text/plain", "Content-Type": "application/x-www-form-urlencoded"}

username = "" # user's mail
password = "" # user's password

r = requests.post(url, data={"username": username, "password": password}, headers=headers)
print("status:", r.status_code)
print("body:", r.text)

status: 201
body: TGT-14002163-WPJa869YuH5TwKRA1VSXo6JbSzeiD72fX-nRP3tku9-YqS6GzgM9juYpe--eZWFt-D0-cas-7dc76c888-d9cdw


In [5]:
tgt = "TGT-14002163-WPJa869YuH5TwKRA1VSXo6JbSzeiD72fX-nRP3tku9-YqS6GzgM9juYpe--eZWFt-D0-cas-7dc76c888-d9cdw"

url = "https://seffaflik.epias.com.tr/electricity-service/v1/markets/dam/data/mcp"
headers = {
    "TGT": tgt,
    "Accept-Language": "en",
    "Accept": "application/json",
    "Content-Type": "application/json",
}
body = {
    "startDate": "2024-01-01T00:00:00+03:00",
    "endDate":   "2024-01-03T00:00:00+03:00",
}

r = requests.post(url, headers=headers, json=body)
print(r.status_code)
print(r.json() if r.headers.get("Content-Type","").startswith("application/json") else r.text)

200
{'items': [{'date': '2024-01-01T00:00:00+03:00', 'hour': '00:00', 'price': 1299.98, 'priceUsd': 44.16, 'priceEur': 39.91}, {'date': '2024-01-01T01:00:00+03:00', 'hour': '01:00', 'price': 1299.98, 'priceUsd': 44.16, 'priceEur': 39.91}, {'date': '2024-01-01T02:00:00+03:00', 'hour': '02:00', 'price': 1248.54, 'priceUsd': 42.41, 'priceEur': 38.33}, {'date': '2024-01-01T03:00:00+03:00', 'hour': '03:00', 'price': 1299.98, 'priceUsd': 44.16, 'priceEur': 39.91}, {'date': '2024-01-01T04:00:00+03:00', 'hour': '04:00', 'price': 1200, 'priceUsd': 40.76, 'priceEur': 36.84}, {'date': '2024-01-01T05:00:00+03:00', 'hour': '05:00', 'price': 1200, 'priceUsd': 40.76, 'priceEur': 36.84}, {'date': '2024-01-01T06:00:00+03:00', 'hour': '06:00', 'price': 1248.54, 'priceUsd': 42.41, 'priceEur': 38.33}, {'date': '2024-01-01T07:00:00+03:00', 'hour': '07:00', 'price': 1299.99, 'priceUsd': 44.16, 'priceEur': 39.91}, {'date': '2024-01-01T08:00:00+03:00', 'hour': '08:00', 'price': 1299.99, 'priceUsd': 44.16, 'pr

In [7]:
PTF_URL = "https://seffaflik.epias.com.tr/electricity-service/v1/markets/dam/data/mcp"

def month_start_end_strings(year: int, month: int):
    start = datetime(year, month, 1)
    if month == 12:
        next_month = datetime(year + 1, 1, 1)
    else:
        next_month = datetime(year, month + 1, 1)
    last_day = next_month - timedelta(days=1)
    return (
        start.strftime("%Y-%m-%dT00:00:00+03:00"),
        last_day.strftime("%Y-%m-%dT00:00:00+03:00"),
    )

def epias_post(url: str, body: dict, tgt: str):
    headers = {
        "TGT": tgt,
        "Accept-Language": "en",
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    r = requests.post(url, headers=headers, json=body, timeout=60)
    r.raise_for_status()
    return r.json()

all_items = []
for m in range(1, 13):
    start_str, end_str = month_start_end_strings(2024, m)
    body = {"startDate": start_str, "endDate": end_str}
    data = epias_post(PTF_URL, body, tgt)  # tgt değişkeni hücrende tanımlı olmalı
    items = data.get("items", [])
    all_items.extend(items)

ptf_2024_df = pd.DataFrame(all_items)

# Kronolojik sıraya al
if not ptf_2024_df.empty:
    ts = pd.to_datetime(
        ptf_2024_df["date"].astype(str).str.replace("+03:00", "", regex=False) + " " + ptf_2024_df["hour"].astype(str),
        errors="coerce"
    )
    ptf_2024_df = ptf_2024_df.assign(_ts=ts).sort_values("_ts").drop(columns=["_ts"])

print("Toplam satır:", len(ptf_2024_df))

Toplam satır: 8784


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_3682/1356692021.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(


In [11]:
SMF_URL = "https://seffaflik.epias.com.tr/electricity-service/v1/markets/bpm/data/system-marginal-price"

def month_start_end_strings(year: int, month: int):
    start = datetime(year, month, 1)
    next_month = datetime(year + (month==12), 1 if month==12 else month+1, 1)
    last_day = next_month - timedelta(days=1)
    return (
        start.strftime("%Y-%m-%dT00:00:00+03:00"),
        last_day.strftime("%Y-%m-%dT00:00:00+03:00"),
    )

def epias_post(url: str, body: dict, tgt: str):
    headers = {
        "TGT": tgt,
        "Accept-Language": "en",
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    r = requests.post(url, headers=headers, json=body, timeout=60)
    r.raise_for_status()
    return r.json()

smf_items = []
for m in range(1, 13):
    s, e = month_start_end_strings(2024, m)
    data = epias_post(SMF_URL, {"startDate": s, "endDate": e}, tgt)
    smf_items.extend(data.get("items", []))

smf_2024_df = pd.DataFrame(smf_items)

if not smf_2024_df.empty:
    ts = pd.to_datetime(
        smf_2024_df["date"].astype(str).str.replace("+03:00", "", regex=False) + " " + smf_2024_df["hour"].astype(str),
        errors="coerce"
    )
    smf_2024_df = smf_2024_df.assign(_ts=ts).sort_values("_ts").drop(columns=["_ts"])

print("SMF satır:", len(smf_2024_df))

/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_3682/1765372984.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(


SMF satır: 8784


In [15]:
pp_path = "/Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/GAIN ENERGY/pp_list.json"

with open(pp_path, "r", encoding="utf-8") as f:
    raw_pp = json.load(f)

def normalize_pp(pp: dict):
    return {
        "powerPlantName": pp.get("powerPlantName") or pp.get("powerplantName"),
        "organizationId": pp.get("organizationId"),
        "powerPlantId": pp.get("powerPlantId") or pp.get("powerplantId"),
        "uevcbId": pp.get("uevcbId"),
    }

pp_list = [normalize_pp(pp) for pp in raw_pp]

for i, pp in enumerate(pp_list, 1):
    print(i, pp)

1 {'powerPlantName': 'MASLAKTEPE RES', 'organizationId': 12717, 'powerPlantId': 2468, 'uevcbId': 3207214}
2 {'powerPlantName': 'EBER RES', 'organizationId': 12517, 'powerPlantId': 2316, 'uevcbId': 3217197}
3 {'powerPlantName': 'YANBOLU HES', 'organizationId': 8801, 'powerPlantId': 1884, 'uevcbId': 2813560}
4 {'powerPlantName': 'MELİKOM HES', 'organizationId': 9709, 'powerPlantId': 2142, 'uevcbId': 3196990}


In [25]:
KGUP_URL = "https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/dpp-first-version"
REGION = "TR1"

selected_idx = [1, 2, 3, 4]
selected_pp = [pp_list[i-1] for i in selected_idx]

def fetch_kgup_for_pp(pp: dict, year: int = 2024, delay: float = 0.2) -> pd.DataFrame:
    rows = []
    for m in range(1, 13):
        s, e = month_start_end_strings(year, m)
        body = {
            "startDate": s,
            "endDate": e,
            "organizationId": pp["organizationId"],
            "uevcbId": pp["uevcbId"],
            "region": REGION,
        }
        data = epias_post(KGUP_URL, body, tgt)
        for it in data.get("items", []):
            rows.append({
                "date": it.get("date"),          # tam ISO datetime (2024-06-03T19:00:00+03:00)
                "hour": it.get("time"),          # API’den dönen 'time' alanını hour’a yaz
                "KGUP": it.get("toplam"),
                "powerPlantName": pp["powerPlantName"],
                "organizationId": pp["organizationId"],
                "uevcbId": pp["uevcbId"],
                "powerPlantId": pp["powerPlantId"],
            })
        time.sleep(delay)

    df = pd.DataFrame(rows)
    if not df.empty:
        # 'date' kolonundan sadece gün kısmını al
        date_day = df["date"].astype(str).str.split("T").str[0]
        # 'hour' kolonuyla birleştirip datetime üret
        ts = pd.to_datetime(date_day + " " + df["hour"].astype(str), errors="coerce")
        df = (
            df.assign(_ts=ts)
              .sort_values(["powerPlantName", "_ts"])
              .drop(columns=["_ts"])
              .reset_index(drop=True)
        )
    return df

# Her santral için ayrı DF: iki sözlük (isimle ve uevcbId ile erişim)
kgup_by_name = {}
kgup_by_uevcb = {}

for pp in selected_pp:
    df_pp = fetch_kgup_for_pp(pp)
    kgup_by_name[pp["powerPlantName"]] = df_pp
    kgup_by_uevcb[pp["uevcbId"]] = df_pp
    print(pp["powerPlantName"], "satır:", len(df_pp))

# Örnek erişim:
# kgup_by_name["MASLAKTEPE RES"].head()
# kgup_by_uevcb[3207214].head()

MASLAKTEPE RES satır: 8784
EBER RES satır: 8784
YANBOLU HES satır: 8784
MELİKOM HES satır: 8784


In [27]:
URETIM_URL = "https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/realtime-generation"

# Seçeceğin santrallerin indekslerini düzenle (ör: [1, 3])
selected_idx = [1, 2, 3, 4]
selected_pp = [pp_list[i-1] for i in selected_idx]

def fetch_generation_for_pp(pp: dict, year: int = 2024, delay: float = 0.2) -> pd.DataFrame:
    rows = []
    for m in range(1, 13):
        s, e = month_start_end_strings(year, m)  # mevcut fonksiyonunu kullanıyor
        body = {
            "startDate": s,
            "endDate": e,
            "powerPlantId": pp["powerPlantId"],
        }
        data = epias_post(URETIM_URL, body, tgt)  # mevcut fonksiyonunu ve tgt'yi kullanıyor
        for it in data.get("items", []):
            rows.append({
                "date": it.get("date"),
                "hour": it.get("time"),       # API 'time' döner; birleşik isim için 'hour'a yaz
                "URETIM": it.get("total"),    # gerçekleşen üretim (MWh)
                "powerPlantName": pp["powerPlantName"],
                "organizationId": pp["organizationId"],
                "uevcbId": pp["uevcbId"],
                "powerPlantId": pp["powerPlantId"],
            })
        time.sleep(delay)
    df = pd.DataFrame(rows)
    if not df.empty:
        date_day = df["date"].astype(str).str.split("T").str[0]
        ts = pd.to_datetime(date_day + " " + df["hour"].astype(str), errors="coerce")
        df = df.assign(_ts=ts).sort_values("_ts").drop(columns=["_ts"]).reset_index(drop=True)
    return df

# Her santral için ayrı DF sözlükleri
uretim_by_name = {}
uretim_by_ppid = {}

for pp in selected_pp:
    df_pp = fetch_generation_for_pp(pp)
    uretim_by_name[pp["powerPlantName"]] = df_pp
    uretim_by_ppid[pp["powerPlantId"]] = df_pp
    print(pp["powerPlantName"], "satır:", len(df_pp))

# Örnek erişim:
# uretim_by_name["MELİKOM HES"].head()
# uretim_by_ppid[2142].head()

/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_3682/2185093498.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(date_day + " " + df["hour"].astype(str), errors="coerce")


MASLAKTEPE RES satır: 8784


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_3682/2185093498.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(date_day + " " + df["hour"].astype(str), errors="coerce")


EBER RES satır: 8784


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_3682/2185093498.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(date_day + " " + df["hour"].astype(str), errors="coerce")


YANBOLU HES satır: 8784
MELİKOM HES satır: 8775


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_3682/2185093498.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(date_day + " " + df["hour"].astype(str), errors="coerce")


In [29]:
def ensure_hour_and_sort(df: pd.DataFrame) -> pd.DataFrame:
    # hour yoksa veya None/boş ise date'ten "HH:MM" türet
    hour_src = None
    if "hour" in df.columns:
        hour_src = df["hour"].astype(str)
    elif "time" in df.columns:
        hour_src = df["time"].astype(str)
    else:
        hour_src = pd.Series([None] * len(df))

    derived = df["date"].astype(str).str.split("T").str[1].str[:5]  # "HH:MM"
    df = df.copy()
    df["hour"] = hour_src
    mask = df["hour"].isna() | (df["hour"] == "None") | (df["hour"].str.strip() == "") | (df["hour"] == "NaN")
    df.loc[mask, "hour"] = derived[mask]

    # kronolojik sıraya al
    date_day = df["date"].astype(str).str.split("T").str[0]
    ts = pd.to_datetime(date_day + " " + df["hour"].astype(str), errors="coerce")
    df = df.assign(_ts=ts).sort_values("_ts").drop(columns=["_ts"]).reset_index(drop=True)
    return df

# KGÜP için (sözlük: isim -> DF)
if 'kgup_by_name' in globals():
    for name, d in kgup_by_name.items():
        kgup_by_name[name] = ensure_hour_and_sort(d)
        print(f"KGÜP | {name}: hour null sayısı -> {kgup_by_name[name]['hour'].isna().sum()}")

# Üretim için (sözlük: isim -> DF)
if 'uretim_by_name' in globals():
    for name, d in uretim_by_name.items():
        uretim_by_name[name] = ensure_hour_and_sort(d)
        print(f"ÜRETİM | {name}: hour null sayısı -> {uretim_by_name[name]['hour'].isna().sum()}")

# Tek bir DF varsa örnek:
# kgup_2024_df = ensure_hour_and_sort(kgup_2024_df)
# uretim_2024_df = ensure_hour_and_sort(uretim_2024_df)

KGÜP | MASLAKTEPE RES: hour null sayısı -> 0
KGÜP | EBER RES: hour null sayısı -> 0
KGÜP | YANBOLU HES: hour null sayısı -> 0
KGÜP | MELİKOM HES: hour null sayısı -> 0
ÜRETİM | MASLAKTEPE RES: hour null sayısı -> 0
ÜRETİM | EBER RES: hour null sayısı -> 0
ÜRETİM | YANBOLU HES: hour null sayısı -> 0
ÜRETİM | MELİKOM HES: hour null sayısı -> 0


In [32]:
BASE_DIR = "/Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data"
os.makedirs(BASE_DIR, exist_ok=True)

def slugify(name):
    s = str(name).strip().replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9_\-]", "", s)

def save_df(df: pd.DataFrame, path: str, label: str):
    if not isinstance(df, pd.DataFrame) or df.empty:
        print(f"Skip (empty): {label}")
        return
    df.to_csv(path, index=False)
    print(f"Saved {len(df)} rows -> {path}")

# PTF / SMF (tek set)
if 'ptf_2024_df' in globals(): save_df(ptf_2024_df, f"{BASE_DIR}/ptf_2024.csv", "PTF")
if 'smf_2024_df' in globals(): save_df(smf_2024_df, f"{BASE_DIR}/smf_2024.csv", "SMF")

# KGÜP (santral bazlı)
if 'kgup_by_name' in globals() and isinstance(kgup_by_name, dict):
    for name, df in kgup_by_name.items():
        uev = df['uevcbId'].iloc[0] if 'uevcbId' in df.columns and not df.empty else "unknown"
        fn = f"kgup_2024_{uev}_{slugify(name)}.csv"
        save_df(df, f"{BASE_DIR}/{fn}", f"KGUP/{name}")

# Üretim (santral bazlı)
if 'uretim_by_name' in globals() and isinstance(uretim_by_name, dict):
    for name, df in uretim_by_name.items():
        ppid = df['powerPlantId'].iloc[0] if 'powerPlantId' in df.columns and not df.empty else "unknown"
        fn = f"uretim_2024_{ppid}_{slugify(name)}.csv"
        save_df(df, f"{BASE_DIR}/{fn}", f"URETIM/{name}")

Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/ptf_2024.csv
Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/smf_2024.csv
Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/kgup_2024_3207214_MASLAKTEPE_RES.csv
Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/kgup_2024_3217197_EBER_RES.csv
Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/kgup_2024_2813560_YANBOLU_HES.csv
Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/kgup_2024_3196990_MELKOM_HES.csv
Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/uretim_2024_2468_MASLAKTEPE_RES.csv
Saved 8784 rows -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/uretim_2024_2316_EBER_RES.c

In [7]:
from pandas import DatetimeTZDtype

def make_naive_dt(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "datetime" in df.columns:
        df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")
        if isinstance(df["datetime"].dtype, DatetimeTZDtype):
            df["datetime"] = df["datetime"].dt.tz_localize(None)
    return df

In [8]:
from glob import glob

BASE = "/Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data"
os.makedirs(f"{BASE}/out", exist_ok=True)

def slugify(s):
    s = str(s).strip().replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9_\-]", "", s)

def build_datetime(df, date_col="date", hour_col="hour"):
    df = df.copy()
    # hour yoksa/date içinden türet
    if hour_col not in df.columns or df[hour_col].isna().any():
        hhmm = df[date_col].astype(str).str.split("T").str[1].str[:5]
        if hour_col not in df.columns:
            df[hour_col] = hhmm
        else:
            mask = df[hour_col].isna() | (df[hour_col].astype(str).isin(["", "None", "NaN"]))
            df.loc[mask, hour_col] = hhmm[mask]
    day = df[date_col].astype(str).str.split("T").str[0]
    dt = pd.to_datetime(day + " " + df[hour_col].astype(str), errors="coerce")
    return df.assign(datetime=dt, Tarih=day, Saat=df[hour_col].astype(str))

# PTF
ptf = pd.read_csv(f"{BASE}/ptf_2024.csv")
ptf = build_datetime(ptf)
ptf = ptf.rename(columns={"price":"PTF"})[["datetime","PTF"]]
ptf = make_naive_dt(ptf)

# SMF
smf = pd.read_csv(f"{BASE}/smf_2024.csv")
smf = build_datetime(smf)
smf = smf.rename(columns={"systemMarginalPrice":"SMF"})[["datetime","SMF"]]
smf = make_naive_dt(smf)

# Santral meta (ID’lerden dosya bulmak için)
plants = [
    {"name":"MASLAKTEPE RES", "uevcbId":3207214, "powerPlantId":2468},
    {"name":"EBER RES",       "uevcbId":3217197, "powerPlantId":2316},
    {"name":"YANBOLU HES",    "uevcbId":2813560, "powerPlantId":1884},
    {"name":"MELİKOM HES",    "uevcbId":3196990, "powerPlantId":2142},
]

def first_or_none(pattern):
    files = glob(pattern)
    return files[0] if files else None

for meta in plants:
    name, uev, ppid = meta["name"], meta["uevcbId"], meta["powerPlantId"]

    kgup_path   = first_or_none(f"{BASE}/kgup_2024_{uev}_*.csv")
    uretim_path = first_or_none(f"{BASE}/uretim_2024_{ppid}_*.csv")
    if not kgup_path or not uretim_path:
        print(f"Skip (missing files): {name} | KGÜP={bool(kgup_path)} ÜRETİM={bool(uretim_path)}")
        continue

    # KGÜP
    kgup = pd.read_csv(kgup_path)
    kgup = build_datetime(kgup)
    kgup = kgup.rename(columns={"KGUP":"KGUP"})[["datetime","Tarih","Saat","KGUP"]]

    # Üretim
    uretim = pd.read_csv(uretim_path)
    uretim = build_datetime(uretim)
    uretim = uretim.rename(columns={"URETIM":"URETIM"})[["datetime","URETIM"]]

    # KGÜP ⨝ Üretim (inner) + datetime’ları tz-naive yap
    df = kgup.merge(uretim, on="datetime", how="inner")
    df  = make_naive_dt(df)

    # PTF/SMF ekle (left)
    df = df.merge(ptf, on="datetime", how="left").merge(smf, on="datetime", how="left")

    # Ay
    df["Ay"] = df["datetime"].dt.month

    # Poz/Neg dengesizlik fiyatları
    df["Poz_DF"] = (df[["PTF","SMF"]].min(axis=1) * 0.97).astype(float)
    df["Neg_DF"] = (df[["PTF","SMF"]].max(axis=1) * 1.03).astype(float)

    # Dengesizlik ve gelirler
    df["KGUP"] = df["KGUP"].astype(float)
    df["URETIM"] = df["URETIM"].astype(float)
    df["PTF"] = df["PTF"].astype(float)
    df["SMF"] = df["SMF"].astype(float)

    df["Dengesizlik"] = (df["URETIM"] - df["KGUP"])
    df["GOP_Geliri"] = (df["KGUP"] * df["PTF"])
    df["Dengesizlik_Tutarı"] = df.apply(
        lambda r: r["Dengesizlik"] * (r["Poz_DF"] if r["Dengesizlik"] >= 0 else r["Neg_DF"]),
        axis=1
    )
    df["Net_Gelir"] = (df["GOP_Geliri"] + df["Dengesizlik_Tutarı"])

    # Dengesizlik Maliyeti (negatif olamaz)
    df["Dengesizlik_Maliyeti"] = ((df["URETIM"] * df["PTF"]) - df["Net_Gelir"]).clip(lower=0)

    # Birim DM (saatlik)
    df["Birim_DM"] = df.apply(lambda r: (r["Dengesizlik_Maliyeti"] / r["URETIM"]) if r["URETIM"] not in (0, 0.0) else 0.0, axis=1)

    # Çıktı sırası
    out_cols = [
        "Tarih","Ay","Saat",
        "PTF","SMF","Poz_DF","Neg_DF",
        "KGUP","URETIM","Dengesizlik",
        "GOP_Geliri","Dengesizlik_Tutarı","Net_Gelir",
        "Dengesizlik_Maliyeti","Birim_DM"
    ]
    df_out = df[out_cols].copy()

    out_path = f"{BASE}/out/{slugify(name)}_birlesik_2024.csv"
    df_out.to_csv(out_path, index=False)
    print(f"Saved -> {out_path} ({len(df_out)} rows)")

/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_6900/2079064281.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt = pd.to_datetime(day + " " + df[hour_col].astype(str), errors="coerce")


Saved -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/out/MASLAKTEPE_RES_birlesik_2024.csv (8784 rows)
Saved -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/out/EBER_RES_birlesik_2024.csv (8784 rows)
Saved -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/out/YANBOLU_HES_birlesik_2024.csv (8784 rows)
Saved -> /Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/out/MELKOM_HES_birlesik_2024.csv (8775 rows)


In [9]:
# Kayıp satırları analiz et
kgup_dates = set(kgup['datetime'].dt.strftime('%Y-%m-%d %H:%M'))
uretim_dates = set(uretim['datetime'].dt.strftime('%Y-%m-%d %H:%M'))

missing_in_kgup = uretim_dates - kgup_dates
missing_in_uretim = kgup_dates - uretim_dates

print(f"KGÜP'te eksik saatler: {len(missing_in_kgup)}")
print(f"Üretim'de eksik saatler: {len(missing_in_uretim)}")
if missing_in_kgup:
    print("KGÜP'te eksik saatler:", sorted(list(missing_in_kgup))[:5])
if missing_in_uretim:
    print("Üretim'de eksik saatler:", sorted(list(missing_in_uretim))[:5])

KGÜP'te eksik saatler: 0
Üretim'de eksik saatler: 9
Üretim'de eksik saatler: ['2024-02-24 03:00', '2024-03-13 05:00', '2024-08-07 21:00', '2024-08-07 22:00', '2024-08-07 23:00']
